In [1]:
COMMANDLINE = False

In [2]:
if not COMMANDLINE:
    # go wide screen
    from IPython.core.display import display, HTML
    display(HTML("<style>.container { width:100% !important; }</style>"))

    # Next two lines force automatic reload of loaded module. Convenient if
    # one is still fiddeling with them.
    %load_ext autoreload
    %autoreload 2

In [3]:
!mv ~/srebin.py .

mv: cannot stat ‘/home1/04287/mxhf/srebin.py’: No such file or directory


In [4]:
import sys
sys.path.append('/home1/04287/mxhf/HETDEX_API/')

In [5]:
if not COMMANDLINE:
    %matplotlib inline
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm
    from astropy.visualization import ZScaleInterval

import astropy.units as u
from astropy.coordinates import SkyCoord

from hetdex_api.shot import *

In [6]:
from __future__ import print_function

import pylab
from astropy.io import fits
import sys
from scipy import *
from scipy import optimize
import spectrum
import numpy
import optparse
from matplotlib import pyplot as plt
import spectrum
from astropy.io import ascii
from astropy.table import Table
import os
import glob
import numpy as np

from scipy.interpolate import UnivariateSpline

import srebin
import sys
from astropy.table import Column, vstack
from astropy.io import ascii
import argparse
import pickle
import glob

In [7]:
import numpy as np

In [8]:
def get_rebinned(ww, ff, start = 3494.74, step =  1.9858398, stop = 5500.):
    N = int( np.ceil( (stop - start)/step ) )
    _lw, _lf = srebin.linlin(ww, ff, start, step, stop, dowarn=False)
    
    lw = (np.arange(N) * step) + start
    lf = np.zeros_like(lw)
    
    lf[:min(N, len(_lf))] = _lf[:min(N, len(_lf))]

    if not _lw[0] == lw[0]:
        print("Binning is broken, please check.")
        return np.nan
    
    return lw, lf

In [9]:
def rebin_IFU(t, p):
    expnum, ifuslot = p
    print("Processing expnum {} ifuslot {} ".format(expnum, ifuslot.decode("utf-8")) )
    amplifiers = \
        np.unique(np.unique( [ x['amp'] for x in t.where("""(expnum == {expnum}) & (ifuslot == {ifuslot})""".format(expnum=expnum, ifuslot=ifuslot))] ))

    for amp in amplifiers:
        rebin_path = "{}/exp{:02d}".format(shot, int(expnum))

        rebin_filename = "multi_xxx_{ifuslot}_xxx_{amp}_rebin.pickle".format(ifuslot=ifuslot.decode("utf-8"),amp=amp.decode("utf-8"))
        rebin_file_path = os.path.join(rebin_path,rebin_filename)

        if os.path.exists( rebin_file_path ) and not force_rebin:
            print("{} already exists, skipping ...".format(rebin_file_path))
            continue
        else:    
            rebinned = {}

            idx = [ x['fibidx'] for x in t.where( """(expnum == {expnum}) & (ifuslot == {ifuslot}) & (amp == {amp})""".format(expnum=expnum, ifuslot=ifuslot, amp=amp) ) ]
            www = [ x["wavelength"] for x in t.where( """(expnum == {expnum}) & (ifuslot == {ifuslot}) & (amp == {amp})""".format(expnum=expnum, ifuslot=ifuslot, amp=amp) ) ]

            for ext in extensions:
                fff = [ x[ext] for x in t.where( """(expnum == {expnum}) & (ifuslot == {ifuslot}) & (amp == {amp})""".format(expnum=expnum, ifuslot=ifuslot, amp=amp) ) ]
                data = []
                print("Rebinning expnum {} , ifuslot {} , amp {} , ext {} ".format( expnum, ifuslot.decode("utf-8"), amp.decode("utf-8"), ext) )

                for i,ww,ff in zip(idx,www,fff):

                    lw, lf = get_rebinned(ww, ff, start = 3494.74, step =  1.9858398, stop = 5500.)
                    data.append(lf)

                rebinned[ext] = np.array(data)

            try:
                os.makedirs(rebin_path)
            except:
                pass

            with open(rebin_file_path, 'wb') as f:
                # Pickle the 'data' dictionary using the highest protocol available.
                print("Writing ", rebin_file_path)
                pickle.dump((lw, rebinned), f, pickle.HIGHEST_PROTOCOL)
                #sys.exit(1)
    return True
                    

In [10]:
shot = '20180124v010'
NCPU = 20
force_rebin = False

#ifuslots = [b'022']
extensions = ["spectrum", "sky_subtracted", "sky_spectrum", "fiber_to_fiber"]


filename = '/work/03946/hetdex/hdr1/reduction/data/{}.h5'.format(shot)
fileh = tb.open_file(filename)
t = fileh.root.Data.Fibers


In [14]:
exposures = [1,2,3]                  
# find which ifuslots we have
ifuslots = np.unique(np.unique(  [x['ifuslot'] for x in t]  ))

ifuslots = ifuslots[:2] # for testing

# create array with every exposure/ifu combination to map multiprocessing to
for i in ifuslots:
    for e in exposures:
        exp_ifu.append([e,i])
        rebin_IFU(t, exp_ifu[2])
    

Processing expnum 3 ifuslot 022 
20180124v010/exp03/multi_xxx_022_xxx_LL_rebin.pickle already exists, skipping ...
20180124v010/exp03/multi_xxx_022_xxx_LU_rebin.pickle already exists, skipping ...
20180124v010/exp03/multi_xxx_022_xxx_RL_rebin.pickle already exists, skipping ...
20180124v010/exp03/multi_xxx_022_xxx_RU_rebin.pickle already exists, skipping ...


True

# Check against previous rebinned file

In [ ]:
!ls /work/04287/mxhf/maverick/hetdex/cubes/exp01/multi_???_022_???_LL_rebin.pickle

In [48]:
ww,rebinned = pickle.load(open("/work/04287/mxhf/maverick/hetdex/cubes/rebin/20180124v010/exp01/multi_317_022_039_LL_rebin.pickle",'rb') , encoding='latin1')

In [49]:
rebinned['sky_spectrum']

array([[ 44.16650608,  40.28904142,  39.14809976, ...,   0.        ,
          0.        ,   0.        ],
       [ 45.00337588,  40.60051559,  39.85620886, ..., 143.97700669,
          0.        ,   0.        ],
       [ 45.358628  ,  40.59465581,  40.1559046 , ..., 144.80466677,
          0.        ,   0.        ],
       ...,
       [ 44.70013365,  41.43942216,  39.75027431, ..., 162.82325068,
        163.82772024, 168.67576862],
       [ 43.5854706 ,  40.4234008 ,  38.7523115 , ..., 158.85326402,
        159.82116533, 164.55744557],
       [ 43.84256613,  40.67584297,  38.96999006, ..., 159.16458694,
        160.07483788, 164.90503294]])

In [50]:
nww,nrebinned = pickle.load(open("20180124v010/exp01/multi_xxx_022_xxx_LL_rebin.pickle",'rb') , encoding='latin1')

In [51]:
nrebinned['sky_spectrum']

array([[ 44.16650608,  40.28904142,  39.14809976, ...,   0.        ,
          0.        ,   0.        ],
       [ 45.00337588,  40.60051559,  39.85620886, ..., 143.97700669,
          0.        ,   0.        ],
       [ 45.358628  ,  40.59465581,  40.1559046 , ..., 144.80466677,
          0.        ,   0.        ],
       ...,
       [ 44.70013365,  41.43942216,  39.75027431, ..., 162.82325068,
        163.82772024, 168.67576862],
       [ 43.5854706 ,  40.4234008 ,  38.7523115 , ..., 158.85326402,
        159.82116533, 164.55744557],
       [ 43.84256613,  40.67584297,  38.96999006, ..., 159.16458694,
        160.07483788, 164.90503294]])

In [58]:
difference = rebinned['sky_spectrum'] - nrebinned['sky_spectrum']
np.std(difference) 

0.0

In [61]:
from hetdex_api.survey import Survey
survey = Survey('hdr1')
thdr1 = Table(survey.hdfile.root.Survey[:])


In [67]:
ii  = thdr1["shotid"] >= 20181100000
ii *= thdr1["shotid"] <= 20190100000

In [68]:
np.sum(ii)

251

In [69]:
251*250/60.

1045.8333333333333

In [72]:
3 * 50 * 4 * 250 * 2

300000